In [10]:
import pandas as pd
import requests
import json

In [ ]:
events = []

eventsFilePath = "events/events.json"

with open(eventsFilePath, "r") as f:
    events_data = json.load(f)

events = events_data["pageProps"]["events"]

print("Event IDs and Names:")
print("====================")

vctEvents = [event for event in events if event["shortName"].startswith("VCT")]

vctEventIds = [event["id"] for event in vctEvents]

print("VCT Event IDs:")
print(vctEventIds)
print(f"\nTotal VCT events found: {len(vctEventIds)}")


Event IDs and Names:
VCT Event IDs:
[5574, 5461, 5469, 5465, 5474, 5351, 5232, 5228, 5230, 5233]

Total VCT events found: 10


In [ ]:
all_match_events = []
all_event_locations = []
all_player_economies = []

def fetchEventData(eventId):
    url = f"https://be-prod.rib.gg/v1/series?completed=true&take=50&eventIds%5B%5D={eventId}"
    JSONContent = requests.get(url).json()
    matchIDs = []
    matchMaps = []
    
    matchEvents = []
    eventLocations = []
    playerEconomies = []
    
    for data in JSONContent["data"]:
        for match in data["matches"]:
            matchIDs.append(match["id"])
            matchMaps.append(match["map"])
    
    for matchID, matchMap in zip(matchIDs, matchMaps):
        content = requests.get("https://be-prod.rib.gg/v1/matches/" + str(matchID) + "/details").json()
        for event in content["events"]:
            matchEvents.append([matchMap["name"],
                                content["id"],
                                event["roundId"],
                                event["roundNumber"],
                                event["roundTimeMillis"],
                                event["eventType"],
                                event["killId"],
                                event["tradedByKillId"],
                                event["tradedForKillId"],
                                event["bombId"],
                                event["resId"],
                                event["playerId"],
                                event["assists"],
                                event["referencePlayerId"],
                                event["damageType"],
                                event["weaponId"],
                                event["ability"],
                                eventId])

        for location in content["locations"]:
            eventLocations.append([matchMap["name"],
                                location["roundNumber"],
                                location["playerId"],
                                location["roundTimeMillis"],
                                location["locationX"],
                                location["locationY"],
                                location["viewRadians"],
                                eventId])

        for economy in content["economies"]:
            playerEconomies.append([matchMap["name"],
                                    economy["roundId"],
                                    economy["roundNumber"],
                                    economy["playerId"],
                                    economy["agentId"],
                                    economy["score"],
                                    economy["weaponId"],
                                    economy["armorId"],
                                    economy["remainingCreds"],
                                    economy["spentCreds"],
                                    economy["loadoutValue"],
                                    economy["survived"],
                                    economy["kast"],
                                    eventId])
    
    return matchEvents, eventLocations, playerEconomies

In [23]:
for eventId in vctEventIds:
    print(f"Fetching data for event ID: {eventId}")
    match_events, event_locations, player_economies = fetchEventData(eventId)
    all_match_events.extend(match_events)
    all_event_locations.extend(event_locations)
    all_player_economies.extend(player_economies)
    print(f"Data for event ID {eventId} fetched.")

matchEvents_dataset_columns = ["Map", "Match_Id", "Round_ID", "Round_Number", "Round_Time", 
                            "Event_Type", "Kill_ID", "Traded_by_ID", "Traded_for_ID", 
                            "Bomb_Status", "Resurrection_ID", "Player_ID", "Assists", 
                            "Reference_Player_ID", "Damage_Type", "Weapon_ID", "Ability", "Event_ID"]

eventLocations_dataset_columns = ["Map", "Round_Number", "Player_ID", "Round_Time", 
                               "X", "Y", "View_Radians", "Event_ID"]

playerEconomies_dataset_columns = ["Map", "Round_ID", "Round_Number", "Player_ID", "Agent_ID", 
                                "Round_Score", "Weapon_ID", "Armor_ID", "Remaining_Credits", 
                                "Spent_Credits", "Loadout_Value", "Survived", "kast", "Event_ID"]

format_to_csv(all_match_events, matchEvents_dataset_columns, "datasets/valorant-match-events.csv")
format_to_csv(all_event_locations, eventLocations_dataset_columns, "datasets/valorant-event-locations.csv")
format_to_csv(all_player_economies, playerEconomies_dataset_columns, "datasets/valorant-player-economies.csv")

print("All data fetched and saved to consolidated files successfully.")

Fetching data for event ID: 5574
Data for event ID 5574 fetched.
Fetching data for event ID: 5461
Data for event ID 5461 fetched.
Fetching data for event ID: 5469
Data for event ID 5469 fetched.
Fetching data for event ID: 5465
Data for event ID 5465 fetched.
Fetching data for event ID: 5474
Data for event ID 5474 fetched.
Fetching data for event ID: 5351
Data for event ID 5351 fetched.
Fetching data for event ID: 5232
Data for event ID 5232 fetched.
Fetching data for event ID: 5228
Data for event ID 5228 fetched.
Fetching data for event ID: 5230
Data for event ID 5230 fetched.
Fetching data for event ID: 5233
Data for event ID 5233 fetched.
All data fetched and saved to consolidated files successfully.
